## Import part

In [1]:
import pandas as pd

## Data

In [2]:
df=pd.read_csv('df_all_d_new.csv',index_col=[0])
df=df.dropna(axis=0,thresh=50)
df=df.fillna(0)
df

,sofifa_id,physic_21,defending_21,dribbling_21,passing_21,shooting_21,pace_21,release_clause_eur_21,international_reputation_21,skill_moves_21,...,club_contract_valid_until_19,league_level_19,club_team_id_19,weight_kg_19,height_cm_19,age_19,wage_eur_19,value_eur_19,potential_19,overall_19
0,158023,65.0,38.0,95.0,91.0,92.0,85.0,138400000.0,5.0,4.0,...,2021.0,1.0,241.0,72.0,170.0,31.0,575000.0,110500000.0,94.0,94.0
1,188545,82.0,43.0,85.0,78.0,91.0,78.0,132000000.0,4.0,4.0,...,2021.0,1.0,21.0,80.0,184.0,29.0,200000.0,77000000.0,90.0,90.0
2,20801,77.0,35.0,89.0,81.0,93.0,89.0,75900000.0,5.0,5.0,...,2022.0,1.0,45.0,83.0,187.0,33.0,400000.0,77000000.0,94.0,94.0
3,190871,59.0,36.0,94.0,86.0,85.0,91.0,166500000.0,5.0,5.0,...,2022.0,1.0,73.0,68.0,175.0,26.0,300000.0,118500000.0,93.0,92.0
4,192985,78.0,64.0,88.0,93.0,86.0,76.0,161000000.0,4.0,4.0,...,2023.0,1.0,10.0,70.0,181.0,27.0,350000.0,102000000.0,92.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24645,242213,53.0,20.0,55.0,44.0,50.0,62.0,190000.0,1.0,2.0,...,2020.0,4.0,1798.0,69.0,177.0,17.0,2000.0,70000.0,74.0,50.0
24650,244752,40.0,19.0,56.0,40.0,49.0,68.0,150000.0,1.0,2.0,...,2019.0,1.0,112199.0,68.0,173.0,16.0,500.0,100000.0,69.0,52.0
24743,245424,44.0,42.0,55.0,50.0,37.0,59.0,137000.0,1.0,2.0,...,2021.0,1.0,112393.0,65.0,173.0,19.0,2000.0,60000.0,69.0,50.0
24822,241493,61.0,48.0,39.0,30.0,20.0,61.0,143000.0,1.0,2.0,...,2019.0,3.0,1938.0,75.0,185.0,17.0,1000.0,170000.0,70.0,58.0


In [3]:
df_22=pd.read_csv('../raw_data/fifa22/players_22.csv')
df_22=df_22[['sofifa_id','value_eur']]

/var/folders/w7/n5m3c1_n3mn3b92rk5hw7msc0000gn/T/ipykernel_16167/1779480134.py:1: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df_22=pd.read_csv('../raw_data/fifa22/players_22.csv')


In [4]:
new_df=df.merge(df_22,on='sofifa_id',how='inner')
new_df=new_df.fillna(new_df.mean())

## Pipeline

In [ ]:
#pipeline

## XGBoost

In [5]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

/Users/yifeisun/.pyenv/versions/3.8.12/envs/opti_recruit/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [6]:
X=new_df.drop(columns=['sofifa_id','value_eur'])
y=np.log(new_df.value_eur)

In [7]:
X1=new_df.iloc[:,:-1]
y1=new_df.iloc[:,-1]

In [8]:
data_dmatrix=xgb.DMatrix(data=X,label=y)

/Users/yifeisun/.pyenv/versions/3.8.12/envs/opti_recruit/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,
                                               random_state=123)


In [10]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                          colsample_bytree = 0.3, 
                          learning_rate = 0.1,
                          max_depth = 5, 
                          alpha = 10, 
                          n_estimators = 10)

In [11]:
xg_reg.fit(X_train,y_train)

[21:37:47] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=10, n_jobs=8,
             num_parallel_tree=1, objective='reg:linear', predictor='auto',
             random_state=0, reg_alpha=10, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [12]:
y_pred=xg_reg.predict(X_test)

In [13]:
rmse=np.sqrt(mean_squared_error(y_test,y_pred))

In [14]:
rmse

4.95010495407409

In [15]:
predictions={'y_pred':y_pred,
            'y_test':y_test}
show_df=pd.DataFrame(predictions).sort_index()
show_df

,y_pred,y_test
4,11.179546,18.647816
6,11.179546,18.679191
9,11.179546,18.459901
11,11.179546,18.269858
20,11.179546,18.493001
...,...,...
7320,8.828172,12.691580
7327,8.764600,11.775290
7330,8.764600,12.154779
7334,8.764600,11.407565
